<a href="https://colab.research.google.com/github/luca-g97/Interactive-AI-Playground/blob/main/FEN_TrainData_Creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pgn2data
from converter.pgn_data import PGNData
import pandas as pd

pgn_data = PGNData("/content/January2013.pgn")
result = pgn_data.export()

if result.is_complete:

    # read the games file
    games_df = result.get_games_df()
    print(games_df.head())

    # read the moves file
    moves_df = result.get_moves_df()
    print(moves_df.head())

                                game_id  game_order                 event  \
0  8254cbb1-51c4-4e41-b583-d756034101c1           1  Rated Classical game   
1  23df3ead-20eb-47fa-b974-a37d880ac503           2  Rated Classical game   
2  2aa233a2-4264-4ce4-9425-0e268827d7f7           3  Rated Classical game   
3  0786942b-6d66-4b7c-ae04-6f906cfb2ef9           4     Rated Bullet game   
4  03b755b6-19c7-47ac-8fc7-59b7006701c8           5     Rated Bullet game   

                           site date_played round             white  \
0  https://lichess.org/j1dkb5dw  ????.??.??     ?             BFG9k   
1  https://lichess.org/a9tcp02g  ????.??.??     ?    Desmond_Wilson   
2  https://lichess.org/szom2tog  ????.??.??     ?     Kozakmamay007   
3  https://lichess.org/rklpc7mk  ????.??.??     ?  Naitero_Nagasaki   
4  https://lichess.org/1xb3os63  ????.??.??     ?      nichiren1967   

               black result white_elo  ...  winner_loser_elo_diff  eco  \
0            mamalak    1-0      163

In [ ]:
games_df['white_elo'] = pd.to_numeric(games_df['white_elo'], errors='coerce')
games_df['black_elo'] = pd.to_numeric(games_df['black_elo'], errors='coerce')

# Calculate the average elo and filter based on the given conditions
filteredGames = games_df[(games_df[['white_elo', 'black_elo']].mean(axis=1) >= 1500) & (games_df['event'] == 'Rated Classical game')]

# Print or inspect the filtered dataframe
print(filteredGames)

filteredGames.to_csv('Games1500Elo+ - January2013.csv', index=False)

                                   game_id  game_order                 event  \
0     8254cbb1-51c4-4e41-b583-d756034101c1           1  Rated Classical game   
1     23df3ead-20eb-47fa-b974-a37d880ac503           2  Rated Classical game   
2     2aa233a2-4264-4ce4-9425-0e268827d7f7           3  Rated Classical game   
7     f15f3897-10be-4c09-8d30-3844a8ba5533           8  Rated Classical game   
10    17d26264-db93-46b4-8fad-f5274fa43521          11  Rated Classical game   
...                                    ...         ...                   ...   
4168  bbd9f50e-3fd5-4239-8113-a87c68496382        4169  Rated Classical game   
4169  0fce8014-1e27-4acc-bdfb-769ddf32d6b8        4170  Rated Classical game   
4175  26c85045-e6da-4825-94be-1c2104c0f0b7        4176  Rated Classical game   
4176  53455705-770d-4433-be46-4ed64cdb4dd6        4177  Rated Classical game   
4179  e3527a0f-75e8-4b65-842e-122fcaa8f1a5        4180  Rated Classical game   

                              site date

In [ ]:
# Filter the other DataFrame to only include rows where game_id is in df2_filtered
filteredMoves = moves_df[moves_df['game_id'].isin(filteredGames['game_id'])]

# Save the filtered DataFrame to a CSV file
filteredMoves.to_csv('Moves1500Elo+ - January2013.csv', index=False)

In [ ]:
!git clone "https://github.com/SindreSvendby/pgnToFen.git"
!mv /content/pgnToFen/pgntofen.py /content/
import pgntofen

def createFENs(df):
    # Initialize the PGN converter (adjust the initialization as needed)
    pgnConverter = pgntofen.PgnToFen()

    # Initialize variables
    previous_game_id = None
    fen_list = []

    for i, row in df.iterrows():
        current_game_id = row['game_id']
        notation = row['notation']

        if current_game_id != previous_game_id:
            # Start a new game for a new game_id
            pgnConverter = pgntofen.PgnToFen()  # Re-initialize for a new game
            pgnConverter.resetBoard()  # Replace with the actual method to start a new game
            previous_game_id = current_game_id

        # Make the move
        pgnConverter.move(notation)

        # Get the FEN after the move
        fen = pgnConverter.getFullFen()
        fen_list.append(fen)

    return fen_list

# Create a new DataFrame with the necessary columns
temp = filteredMoves.copy()

# Create GameNumber by factorizing the game_id column
temp['Game'] = pd.factorize(temp['game_id'])[0] + 1

# Create DrawNumber by grouping by game_id and counting the rows
temp['Move'] = temp.groupby('game_id').cumcount() + 1

temp['FEN'] = createFENs(temp)

splittedFEN = temp['FEN'].str.split(' ', expand=True)

# Select only the required columns and rename them
trainData = temp[['game_id', 'Game', 'Move', 'FEN']].rename(columns={'game_id': 'GameID'})

# Split the FEN column into multiple columns
splittedFEN = trainData['FEN'].str.split(' ', expand=True)

print(splittedFEN)

# Rename the split columns as needed
splittedFEN.columns = ['Current Field', 'Active Color', 'En Passant', 'Castling Availability']

# Concatenate the original DataFrame with the new split columns
trainData = pd.concat([trainData, splittedFEN], axis=1)

# Save the result to a CSV file
trainData.to_csv('TrainData1500Elo+ - January2013.csv', index=False)

# Print or inspect the resulting DataFrame
print(trainData)

fatal: destination path 'pgnToFen' already exists and is not an empty directory.
mv: cannot stat '/content/pgnToFen/pgntofen.py': No such file or directory
                                                        0  1   2     3
0           rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR  b  e3  KQkq
1         rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR  w   -  KQkq
2         rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBNR  b  d3  KQkq
3       rnbqkbnr/p1pp1ppp/1p2p3/8/3PP3/8/PPP2PPP/RNBQKBNR  w   -  KQkq
4       rnbqkbnr/p1pp1ppp/1p2p3/8/3PP3/P7/1PP2PPP/RNBQ...  b   -  KQkq
...                                                   ... ..  ..   ...
269822             5rk1/5pp1/4p3/3n1bp1/1p6/8/1B3PPP/R5K1  w   -     K
269823           5rk1/5pp1/4p3/3n1bp1/1p6/6P1/1B3P1P/R5K1  b   -     K
269824          5rk1/5pp1/4p3/3n1b2/1p4p1/6P1/1B3P1P/R5K1  w   -     K
269825           5rk1/5pp1/4p3/3n1b2/Rp4p1/6P1/1B3P1P/6K1  b   -     K
269826          2r3k1/5pp1/4p3/3n1b2/Rp4p1/6P1/1B3P1P/6K1  w   